In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from functools import partial

In [2]:
def u0(x):
    """initial condition"""
    u = -torch.sin(torch.pi * x)
    return u

class ffnn(nn.Module):
    """basic FF network for approximating functions"""
    def __init__(self, nn_width=10, num_hidden=2):
        super().__init__()
        
        self.layer_first = nn.Linear(2, nn_width)
        
        layers = []
        for _ in range(num_hidden):
            layers.append(nn.Linear(nn_width, nn_width))
        self.layer_hidden = nn.ModuleList(layers)
        
        self.layer_last = nn.Linear(nn_width, 1)
        
    def forward(self, x, t):
        xt = torch.cat([x, t], dim=1)
        activation = nn.Tanh()
        u = activation(self.layer_first(xt))
        for hidden in self.layer_hidden:
            u = activation(hidden(u))
        u = self.layer_last(u)
        return u

# Create a model

Create a model for
$$
u_t + u u_x = 0
$$
with $u(x,0)=-sin(\pi x)$ and $u(-1,t)=u(1,t)=0$.

In [35]:
# 1. FIXME -- training data
nx_train = 100
nt_train = 100
x1d = torch.linspace(-1,1, nx_train, requires_grad=True)
t1d = torch.linspace(-1,1, nt_train, requires_grad=True)

x, t = torch.meshgrid(x1d, t1d)
x_bc = x[0]
t_bc = 

# 2. set the model
torch.manual_seed(23939)
model = ffnn(15, 2)

# 3. set the optimizer
lr = 0.025
opt = torch.optim.Adam(model.parameters(), lr)
n_epochs = 10000
loss_history = []

# 4. FIXME -- set the loss for BC, IC, and the residual
def loss_function(model, x, t): 
    u = model.forward(x,t)   
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(t), create_graph=True, retain_graph=True)[0]
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(t), create_graph=True, retain_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, create_graph=True, retain_graph=True)[0]
    loss_value = torch.pow((u_t + u*u_x - (0.01/torch.pi) * u_xx),2)
    return loss_value

loss_value_bc = torch.pow(model.forward(-1, t), 2) + torch.pow(model.forward(1, t), 2) + torch.pow(model.forward(x, 0) +  u0(x), 2)

/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [38]:
x[0]

tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], grad_fn=<SelectBackward0>)

In [27]:
x=torch.tensor(np.arange(-1,1,0.02))
t=torch.tensor(np.arange(-1,1,0.02))
X, T = np.meshgrid(x,t)    

In [29]:
T

array([[-1.  , -1.  , -1.  , ..., -1.  , -1.  , -1.  ],
       [-0.98, -0.98, -0.98, ..., -0.98, -0.98, -0.98],
       [-0.96, -0.96, -0.96, ..., -0.96, -0.96, -0.96],
       ...,
       [ 0.94,  0.94,  0.94, ...,  0.94,  0.94,  0.94],
       [ 0.96,  0.96,  0.96, ...,  0.96,  0.96,  0.96],
       [ 0.98,  0.98,  0.98, ...,  0.98,  0.98,  0.98]])

In [30]:
for i in range(n_epochs):

    loss = loss_function(model, X, T)
    opt.zero_grad()
    loss_history.append(loss.item())
    
    loss.backward()
    opt.step()

    if i % 1000 == 0:
        print(f'epoch {i}, loss = {loss}')

TypeError: ignored

# Plot the solution